## Inverse Kinematics: CCD + Angle Limit

In [ ]:
%matplotlib notebook
import numpy as np
import matplotlib.pyplot as plt
from math import tau, pi, sin, cos

In [ ]:
class Arm:
    def __init__(self, ax, ay, length, angle, minAngle=-pi/2, maxAngle=pi/2):
        self.ax = ax
        self.ay = ay
        self.length = length
        self.angle = convTheta(angle)
        self.bx = self.ax + self.length * cos(self.angle)
        self.by = self.ay + self.length * sin(self.angle)
        self.minAngle = convTheta(minAngle)
        self.maxAngle = convTheta(maxAngle)
    
    def angleLimit(self, prevTheta, newTheta):
        theta = convTheta(newTheta - prevTheta)
        if theta < self.minAngle:
            theta = self.minAngle
        elif theta > self.maxAngle:
            theta = self.maxAngle
        self.angle = theta + prevTheta
        self.bx = self.ax + self.length * cos(self.angle)
        self.by = self.ay + self.length * sin(self.angle)

def convTheta(theta): # convert to Angle range between -180 and 180 degrees
    theta = theta % tau
    if theta > pi:
        theta = theta - tau
    return theta 

In [ ]:
N  = 4 # the number of links
OX = 0 # offset x
OY = 0 # offset y

ccd, ccd2 = [], []
linkLength = 1
initTheta = pi / 2

for i in range(N):
    if i == 0:
        ccd.append(Arm(OX, OY, linkLength, initTheta))
        ccd2.append(Arm(OX, OY, linkLength, initTheta, 0, pi))
    else:
        ccd.append(Arm(ccd[i-1].bx, ccd[i-1].by, linkLength, initTheta))
        ccd2.append(Arm(ccd2[i-1].bx, ccd2[i-1].by, linkLength, initTheta, -pi/2, pi/2))
    
CX, CY = [], []        
LX, LY = [], []
for i in range(N):
    CX.append([ccd[i].ax, ccd[i].bx])
    CY.append([ccd[i].ay, ccd[i].by])
    LX.append([ccd2[i].ax, ccd2[i].bx])
    LY.append([ccd2[i].ay, ccd2[i].by])
        
fig = plt.figure(figsize=(5,5))
ax = fig.add_subplot(111, alpha=0.6)
ax.axis([-1,N+1,-1,N+1])
ax.grid()
ax.plot(CX, CY, color='tab:blue', alpha=0.6)
ax.scatter(CX, CY, color='tab:blue', alpha=0.6)
ax.plot(LX, LY, color='magenta', alpha=0.6)
ax.scatter(LX, LY, color='magenta', alpha=0.6)

<img width="70%" align="left" alt="ccd" src="https://user-images.githubusercontent.com/26479204/116737642-671a6c80-aa2c-11eb-898f-0dbf50bddcc9.png">

In [ ]:
def CCD(arm, tx, ty):
    for i in reversed(range(N)):
        targetTheta = np.arctan2(ty - arm[i].ay, tx - arm[i].ax)
        tipTheta = np.arctan2(arm[-1].by - arm[i].ay, arm[-1].bx - arm[i].ax)
        theta = targetTheta - tipTheta;
        for j in range(i, N):
            linkTheta = np.arctan2(arm[j].by - arm[i].ay, arm[j].bx - arm[i].ax)
            linkDist = ((arm[j].bx - arm[i].ax)**2 + (arm[j].by - arm[i].ay)**2)**0.5
            arm[j].bx = arm[i].ax + linkDist * np.cos(theta + linkTheta)
            arm[j].by = arm[i].ay + linkDist * np.sin(theta + linkTheta)
            if j < N - 1:
                arm[j+1].ax = arm[j].bx
                arm[j+1].ay = arm[j].by
    
    PX, PY = [], []
    for i in range(N):
        PX.append(arm[i].ax)
        PY.append(arm[i].ay)
    PX.append(arm[N-1].bx)
    PY.append(arm[N-1].by)
    
    return PX, PY


def CCD2(arm, tx, ty):
    for i in reversed(range(N)):
        targetTheta = np.arctan2(ty - arm[i].ay, tx - arm[i].ax)
        tipTheta = np.arctan2(arm[-1].by - arm[i].ay, arm[-1].bx - arm[i].ax)
        dTheta = convTheta(targetTheta - tipTheta)
        for j in range(i, N):
            if j == 0:
                arm[j].angleLimit(0, arm[j].angle + dTheta)
            else:
                arm[j].ax = arm[j-1].bx
                arm[j].ay = arm[j-1].by
                linkTheta = np.arctan2(arm[j].by - arm[i].ay, arm[j].bx - arm[i].ax)
                linkDist = ((arm[j].bx - arm[i].ax)**2 + (arm[j].by - arm[i].ay)**2)**0.5
                arm[j].bx = arm[i].ax + linkDist * cos(linkTheta + dTheta)
                arm[j].by = arm[i].ay + linkDist * sin(linkTheta + dTheta)
                
                newTheta = np.arctan2(arm[j].by - arm[j].ay, arm[j].bx - arm[j].ax)
                arm[j].angleLimit(arm[j-1].angle, newTheta)

                
    PX, PY = [], []
    for i in range(N):
        PX.append(arm[i].ax)
        PY.append(arm[i].ay)
    PX.append(arm[N-1].bx)
    PY.append(arm[N-1].by)
    
    return PX, PY

tx, ty = N*0.7, N*0.5
CX, CY = CCD(ccd, tx, ty)
LX, LY = CCD2(ccd2, tx, ty)

fig = plt.figure(figsize=(5,5))
ax = fig.add_subplot(111)
ax.axis([-1,N+1,-1,N+1])
ax.grid()
ax.plot(CX, CY, color='tab:blue', alpha=0.6)
ax.scatter(CX, CY, color='tab:blue', alpha=0.6)
ax.plot(LX, LY, color='magenta', alpha=0.6)
ax.scatter(LX, LY, color='magenta', alpha=0.6)
ax.plot([tx], [ty], marker='x', ms=30, color='red')

## Interactive mode: Arm follows Mouse

In [ ]:
def motion(event):
    mx = event.xdata
    my = event.ydata
    Mouse.set_data(mx, my)
    
    CX, CY = CCD(ccd, mx, my)
    CLine.set_data(CX, CY)
    CDot.set_data(CX, CY)
    
    LX, LY = CCD2(ccd2, mx, my)
    LLine.set_data(LX, LY)
    LDot.set_data(LX, LY)
    
    plt.draw()

fig = plt.figure(figsize=(5,5))
ax = fig.add_subplot(111)
ax.axis([-1,N+1,-1,N+1])
ax.grid()

CLine, = ax.plot([],[], linestyle='-', color='tab:blue', alpha=0.6, label='CCD')
CDot, = ax.plot([],[], marker='o', color='tab:blue', alpha=0.6)
LLine, = ax.plot([],[], linestyle='-', color='magenta', alpha=0.6, label='CCD_AL')
LDot, = ax.plot([],[], marker='o', color='magenta', alpha=0.6)
Mouse, = ax.plot([],[], marker='x', ms=20, color='red')

plt.connect('motion_notify_event', motion)
plt.legend(loc='lower right')
plt.show()